In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss
from sklearn.linear_model import LogisticRegression
import os
print(os.listdir("../input"))
%matplotlib inline

['cord-paper-clf']


In [2]:
df_train = pd.read_csv('/kaggle/input/cord-paper-clf/train_w_targets_w_ids_v6.csv')
df_test = pd.read_csv('/kaggle/input/cord-paper-clf/test_wo_targets_w_ids_v6.csv')
df_train.head()

,titles,summaries,terms,ids
0,Continuous Graph Neural Networks,This paper builds on the connection between gr...,"['cs.LG', 'stat.ML']",4c149a8304
1,On Unifying Deep Generative Models,Deep generative models have achieved impressiv...,"['cs.LG', 'stat.ML']",423d33e53d
2,A Flow Base Bi-path Network for Cross-scene Vi...,Drones shooting can be applied in dynamic traf...,['cs.CV'],18e4e85cde
3,Region Similarity Representation Learning,We present Region Similarity Representation Le...,['cs.CV'],a2621fedbd
4,Modeling the Sequence of Brain Volumes by Loca...,We represent the sequence of fMRI (Functional ...,"['cs.LG', 'cs.AI', 'cs.CV']",bf7f0af288


In [3]:
df_train.isna().sum()

titles       20
summaries    56
terms         0
ids           0
dtype: int64

In [4]:
df_train.dropna(inplace=True)

In [5]:
df_train['terms'] = df_train['terms'].apply(lambda x: eval(x))
type(df_train['terms'].iloc[0])

list

In [6]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train.terms)
Y = multilabel_binarizer.transform(df_train.terms)

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df_train.summaries)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

Y.shape, multilabel_binarizer.classes_

((36897, 28),
 array(['cs.AI', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY', 'cs.DC', 'cs.GR',
        'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.MA', 'cs.MM', 'cs.NE',
        'cs.RO', 'cs.SI', 'cs.SY', 'eess.IV', 'eess.SP', 'eess.SY',
        'math.IT', 'math.OC', 'math.ST', 'physics.chem-ph', 'stat.AP',
        'stat.ME', 'stat.ML', 'stat.TH'], dtype=object))

In [7]:
# count_vect = CountVectorizer()
x_test_counts = count_vect.transform(df_test.summaries)

# tfidf_transformer = TfidfTransformer()
x_test_tfidf = tfidf_transformer.transform(x_test_counts)

In [8]:
# sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)

LR = LogisticRegression(random_state=2023)


clf = OneVsRestClassifier(LR)
clf.fit(X_tfidf, Y)
y_pred = clf.predict_proba(x_test_tfidf)

In [9]:
classes = multilabel_binarizer.classes_
for i in range(len(multilabel_binarizer.classes_)):
    df_test[classes[i]] = pd.Series(list(y_pred[:,i]))
    
df_test.drop(columns=['summaries', 'titles'], axis=1, inplace=True)
df_test.head()

,ids,cs.AI,cs.CL,cs.CR,cs.CV,cs.CY,cs.DC,cs.GR,cs.HC,cs.IR,...,eess.SP,eess.SY,math.IT,math.OC,math.ST,physics.chem-ph,stat.AP,stat.ME,stat.ML,stat.TH
0,fd71be9d22,0.036579,0.004236,0.002586,0.973601,0.001680,0.001576,0.015229,0.000895,0.001958,...,0.003109,0.001637,0.000915,0.003260,0.000866,0.000829,0.001085,0.001148,0.026294,0.000866
1,eed2e20fc4,0.831832,0.015116,0.005662,0.011721,0.002216,0.002247,0.002043,0.000793,0.005148,...,0.003166,0.003944,0.001101,0.016535,0.000875,0.000804,0.001080,0.000914,0.195775,0.000875
2,510d5fb335,0.113431,0.007075,0.003137,0.931206,0.001915,0.001610,0.002497,0.001054,0.003452,...,0.004701,0.001619,0.001234,0.003156,0.000954,0.000743,0.001424,0.001116,0.212648,0.000954
3,0bd5f3c9f0,0.054290,0.002772,0.003515,0.849021,0.002367,0.001635,0.003465,0.001002,0.001807,...,0.006609,0.001886,0.000994,0.003875,0.001040,0.000702,0.001583,0.001185,0.274676,0.001040
4,f0ae5d7d05,0.147593,0.003560,0.002881,0.962251,0.001747,0.001155,0.001770,0.000712,0.002172,...,0.001819,0.001299,0.000785,0.002548,0.000897,0.000557,0.000948,0.000730,0.114674,0.000897


In [10]:
df_test.iloc[0]

ids                fd71be9d22
cs.AI                0.036579
cs.CL                0.004236
cs.CR                0.002586
cs.CV                0.973601
cs.CY                 0.00168
cs.DC                0.001576
cs.GR                0.015229
cs.HC                0.000895
cs.IR                0.001958
cs.IT                0.000915
cs.LG                0.081034
cs.MA                0.001968
cs.MM                0.005549
cs.NE                0.005965
cs.RO                0.103756
cs.SI                0.002348
cs.SY                0.001637
eess.IV              0.029857
eess.SP              0.003109
eess.SY              0.001637
math.IT              0.000915
math.OC               0.00326
math.ST              0.000866
physics.chem-ph      0.000829
stat.AP              0.001085
stat.ME              0.001148
stat.ML              0.026294
stat.TH              0.000866
Name: 0, dtype: object

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8034 entries, 0 to 8033
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ids              8034 non-null   object 
 1   cs.AI            8034 non-null   float64
 2   cs.CL            8034 non-null   float64
 3   cs.CR            8034 non-null   float64
 4   cs.CV            8034 non-null   float64
 5   cs.CY            8034 non-null   float64
 6   cs.DC            8034 non-null   float64
 7   cs.GR            8034 non-null   float64
 8   cs.HC            8034 non-null   float64
 9   cs.IR            8034 non-null   float64
 10  cs.IT            8034 non-null   float64
 11  cs.LG            8034 non-null   float64
 12  cs.MA            8034 non-null   float64
 13  cs.MM            8034 non-null   float64
 14  cs.NE            8034 non-null   float64
 15  cs.RO            8034 non-null   float64
 16  cs.SI            8034 non-null   float64
 17  cs.SY         

In [12]:
df_test.to_csv('sample_submission_baseline_5.csv', index=False)

In [13]:
sol = pd.read_csv('/kaggle/input/cord-paper-clf/solution-v7.csv')
sol

,ids,cs.AI,cs.CL,cs.CR,cs.CV,cs.CY,cs.DC,cs.GR,cs.HC,cs.IR,...,eess.SP,eess.SY,math.IT,math.OC,math.ST,physics.chem-ph,stat.AP,stat.ME,stat.ML,stat.TH
0,fd71be9d22,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,eed2e20fc4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,510d5fb335,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0bd5f3c9f0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,f0ae5d7d05,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8029,3020ffad38,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8030,ead9c55cb5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8031,44a1edbb65,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8032,8dd1cbdc8b,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
sol['ids'].equals(df_test['ids'])

True

In [15]:
df_test.columns

Index(['ids', 'cs.AI', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY', 'cs.DC', 'cs.GR',
       'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.MA', 'cs.MM', 'cs.NE', 'cs.RO',
       'cs.SI', 'cs.SY', 'eess.IV', 'eess.SP', 'eess.SY', 'math.IT', 'math.OC',
       'math.ST', 'physics.chem-ph', 'stat.AP', 'stat.ME', 'stat.ML',
       'stat.TH'],
      dtype='object')